In [2]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
from pathlib import Path
import skimage.io
import tensorflow as tf

# 获取项目根目录
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
# os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#基础配置路径
settingsDir = os.path.join(ROOT_DIR,'settings.json')
print (settingsDir)
with open(settingsDir) as f:
    setting = json.load(f)
#获得当前路径
ROOT_DIR = os.getcwd()
sys.path.append(ROOT_DIR)
DEFAULT_LOGS_DIR = Path(os.path.join(os.path.abspath("../"),setting['LOGS_DIR']))
print (DEFAULT_LOGS_DIR)

from mrcnn.config import Config
from mrcnn import utils as utils
from mrcnn import model as modellib
from imgaug import augmenters as iaa

############################################################
#  Configurations
############################################################
# 图像大小
IMGSIZE = (1024, 1024)

class AdrivingConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    #为这个配置命名。例如, 'COCO', 'Experiment 3', 等等。
    NAME = "Adriving"
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    GPU_COUNT = 1
    #每个GPU一次处理多少幅图像。一个12GB的GPU
    #可以处理两幅1024x1024px的图像.
    #根据你的GPU显存和图像大小调整它.
    #使用你的GPU最多能处理的数量可以获得最好的性能.
    IMAGES_PER_GPU = 1
    
    #每个epoch训练的步数
    #这个参数不需要匹配训练集的大小. Tensorboard
    #Tensorboard的更新会在每个epoch的最后保存, so setting this to a
    #所以将这个参数设的小一点，Tensorboard的更新频率更高.
    #在每个epoch的最后也会更新验证的状态，这会花费一些时间
    #所以不要把它设置的过小以避免浪费大量的时间在验证上
    STEPS_PER_EPOCH = 360
    #在每个训练epoch的结尾进行验证的步数.
    #设置大一些可以提升验证的精度, 但是会减慢训练过程.
    VALIDATION_STEPS = 36
    
    #分类数量(包括背景)
    NUM_CLASSES = 1 + 7 
    #图像均值(RGB)
    MEAN_PIXEL = np.array([88.59672608, 95.91837699, 98.90089033])
    
    #Non-max suppression过滤RPN proposals时的阈值.
    #在训练中增大它可以产生更多的propsals.
    RPN_NMS_THRESHOLD = 0.6
    TRAIN_ROIS_PER_IMAGE = 600
    #每幅图像中使用多少anchors来训练RPN
    RPN_TRAIN_ANCHORS_PER_IMAGE = 320
    MAX_GT_INSTANCES = 80
    
    # non-maximum suppression之后保留多少ROIs(训练和预测)
    POST_NMS_ROIS_TRAINING = 4000
    POST_NMS_ROIS_INFERENCE = 2000
    #检测时Non-maximum suppression的阈值
    DETECTION_NMS_THRESHOLD = 0.3
    #接受一个instance的最小可能性
    #ROIs小于该值将被忽略
    DETECTION_MIN_CONFIDENCE = 0.5
    #最终检测的instances的最大数量
    DETECTION_MAX_INSTANCES = 107
    
    #输入图像缩放
    #通常, 训练和预测使用"square" 缩放模式，它在大部分情况下表现的很好
    #在这个模式中, 缩放图像使其短边= IMAGE_MIN_DIM,但是也要确保长边
    #不大于IMAGE_MAX_DIM. 可以对图像填充0使其成为正方形，
    #这样多幅图像可以放在一个batch中
    #可选的缩放模式:
    # none:   无缩放或填充. 返回原图.
    # square: 缩放或填充0，返回[max_dim, max_dim]大小的图像.
    # pad64:  宽和高填充0，使他们成为64的倍数.
    #         如果IMAGE_MIN_DIM 或 IMAGE_MIN_SCALE不为None, 则在填充之前先
    #         缩放. IMAGE_MAX_DIM在该模式中被忽略.
    #         要求为64的倍数是因为在对FPN金字塔的6个levels进行上/下采样时保证平滑(2**6=64).
    # crop:   对图像进行随机裁剪. 首先, 基于IMAGE_MIN_DIM和IMAGE_MIN_SCALE
    #         对图像进行缩放, 然后随机裁剪IMAGE_MIN_DIM x IMAGE_MIN_DIM大小. 
    #         仅在训练时使用.
    IMAGE_MIN_DIM = IMGSIZE[0]
    IMAGE_MAX_DIM = IMGSIZE[1]
    IMAGE_RESIZE_MODE = "none"
    #输出mask的形状
    #更改这个参数需要同时修改neural network mask branch
    MASK_SHAPE = [28, 28]
    
    OPTIMIZER = 'SGD'
    LEARNING_RATE = 1e-6
    EPSILON = 1e-6
    GRADIENT_CLIP_NORM = 5
    ACCUM_ITERS = 1

############################################################
#  Dataset
############################################################
from adriving_util import *

def train(model):
    """Train the model."""
    # Training dataset.
    
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Multiply((0.9, 1.1))
        # iaa.GaussianBlur(sigma=(0.0, 1.0))
    ])
    
    dataset_train = AdrivingDatasetNoResize()
    dataset_train.load_adriving(data_dir, "train", size = IMGSIZE)
    dataset_train.prepare()
    print(len(dataset_train.image_ids))

    # Validation dataset
    dataset_val = AdrivingDatasetNoResize()
    dataset_val.load_adriving(data_dir, "val", size = IMGSIZE)
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    # 通过传入参数layers="heads" 冻结处理head部分的所有层。可以通过传入一个正则表达式选择要训练的层
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                augmentation=augmentation,
                epochs=50,
                layers='heads')
    
    model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            augmentation=augmentation,
            epochs=100,
            layers='4+')
    
    model.train(dataset_train, dataset_val,
        learning_rate=config.LEARNING_RATE/10,
        augmentation=augmentation,
        epochs=150,
        layers='3+')
    # 通过传入参数layers="all"所有层，Fine-tune所有层，上面训练了一会head部分，为了更好的适配新的数据集，需要fine-tune，使用layers='all'参数
    model.train(dataset_train, dataset_val,
        learning_rate=config.LEARNING_RATE/2,
        augmentation=augmentation,
        epochs=100,
        layers='all')
    
    
# --------------------------------------------------------------------------



/media/airobot/airobots/aistars/maskrcnn_lane/settings.json
/media/airobot/airobots/aistars/maskrcnn_lane/logs


In [3]:
############################################################
#  Training
############################################################

if __name__ == '__main__':
    config = AdrivingConfig()
    config.display()
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=os.path.join(DEFAULT_LOGS_DIR, 'mask_rcnn'))
    
    PRETRAIN_WEIGHTS_PATH = os.path.join(ROOT_DIR, setting['MODEL_CHECKPOINT_DIR'], 'mask_rcnn_adriving_0006.h5')

    
    weights_path = PRETRAIN_WEIGHTS_PATH
    model.load_weights(weights_path, by_name=True)
    
    data_dir = Path(os.path.join(ROOT_DIR, setting['TEST_DATA_CLEAN_PATH'], 'train_val'))
    train(model)




Configurations:
ACCUM_ITERS                    1
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        107
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
EPSILON                        1e-06
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                20
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              none
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  1e-06
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES 

FileNotFoundError: [Errno 2] No such file or directory: '/media/airobot/airobots/aistars/maskrcnn_lane/data/train_val/train/image'